In [ ]:
from operator import iadd
from os import listxattr
class RS(BinaryProblem):

    def __init__(self, data):
        super(RS, self).__init__()
        self.data=data# la base
        self.P= len(data.userId.unique()) # nb profil
        self.vMin=10 #min set (Li) longueur min de voisin
        self.maxK=int(self.P/self.vMin)#max k de cluster
        self.vMax=int(self.P-((self.maxK-1)*(self.vMin))) #max Li longueur max de voisin
        self.minK=1#min K de cluster
       


        self.number_of_bits = self.sizeChromosome() #la longueur du chromosome
        self.number_of_objectives = 1
        self.number_of_variables = 1
        self.number_of_constraints = 0

        
        
        self.obj_directions = [self.MAXIMIZE]
        self.obj_labels = ['Ones']

    def sizeChromosome(self):
        x=len(f"{self.maxK:b}")+(self.maxK*((len(f"{self.P:b}")+len(f"{self.vMax:b}")+len(f"{C(self.P,self.P/2):b}"))))  
  
        return x 

    def evaluate(self, solution: BinarySolution) -> BinarySolution:
       #encodage en chaine de caractere avec 0 et 1
       chromosome = ''
       for bits in solution.variables[0]:
            chromosome += '1' if bits else '0'
       #decodage de K 
       K=decodagek(self.maxK,chromosome)
       self.vMax=int(self.P-((K-1)*(self.vMin))) 
       
       couple=getCoupless(self.data,self.P,self.vMin,self.vMax,self.maxK,K,chromosome,similarity_metric='overlap')
       #fonction de fitness 
       solution.objectives[0] =couple
       #score
       return solution

    def create_solution(self) -> BinarySolution:
        new_solution = BinarySolution(number_of_variables=1, number_of_objectives=1)
        new_solution.variables[0] = \
            [True if random.randint(0, 1) == 0 else False for _ in range(self.number_of_bits)]
        return new_solution         
    def get_name(self) -> str:
        return 'RS'
       

#treillis
def getmyset(Ralpha=0,P=0,  ID=0):
      pool = list()
      pool.append(1)
      SOM=0
    
      for j in range(1,Ralpha+1):
          cSOM= SOM
          k=0 
          while (cSOM <= ID):
            
              SOM= cSOM
              c=P-(pool[j-1]+(k))
              d=Ralpha-(j)
              
            
              e=c-d
              if(c<d):
                print('c='+str(c))
                print('d='+str(d))
                print('e='+str(e))
              f=math.factorial(c)
              f=f//math.factorial(d)
              f=f//math.factorial(e)
              cSOM+=f
              if(cSOM<=ID):
                  k=k+1
          pool[j-1]= pool[j-1]+(k)


          if(j-1<Ralpha-1):
              pool.append(pool[j-1]+1)
            
      set2= set();
        
      for x in pool:
          set2.add(x-1)
   # print('set2='+str(set2))    
      return set2
#combinaison
def C(n,r):
    if (r<=n):
       return (math.factorial(n) // (math.factorial(r) * math.factorial(n - r)))
    else:
       return 0      
def getCoupless(data,P,vMin,vMax,maxK,K,solution,similarity_metric='jaccard'):
  c=[]
  T=list(data.userId.unique())# [1,....,610]les profils 
  pos=len(f"{maxK:b}")
  m = data.pivot_table(index=['userId'],columns=['movieId'],values='rating') #la base --> en matrice

  x=len(f"{P:b}")
  y=x+len(f"{vMax:b}")
  z=y+len(f"{C(P,P/2):b}")
  e=[]
  #parcours le chromosome bloc pas bloc avec pos x y z
  for i in range (K):
         
         
         if i< len(solution):
           
           if(vMax!=vMin):
             print('k='+str(K))
             print('actuser='+str(int(solution[pos:pos+x],2) % P))
             #print('cLi='+str(vMin+(int(solution[pos+x:pos+y],2) % (vMax-vMin))))
             #print('cid='+str((int(solution[pos+y:pos+z],2)) % (C(len(T),(vMin+int(solution[pos+x:pos+y],2) % (vMax-vMin)))-1)))
             #c[[active_user,Li(longueur des voisin),id(son id dans le treillis)]]
             ac=int(solution[pos:pos+x],2) % P
             lii=vMin+(int(solution[pos+x:pos+y],2) % (vMax-vMin))
             if(i==0):
               idd=(int(solution[pos+y:pos+z],2)) % (C(len(T)-1,lii)-1)
             else:               
               if(len(T)!=lii):         
                  idd=(int(solution[pos+y:pos+z],2)) % (C(len(T),lii)-1)
               else:
                 idd=0   
  
             print('t='+str(len(T)))
             print('lii        ='+str(lii))
             print('comb(t,lii)='+str(C(len(T),lii)-1))
             print('iddd   =     '+str(idd))
             c.append([ac,lii,idd])
             
             
           else:  #si k==kmax alors vMax==vMin 
             ac=int(solution[pos:pos+x],2) % P
             lii=vMin
             if(i==0):
               idd=(int(solution[pos+y:pos+z],2)) % (C(len(T)-1,lii)-1)
             else:  
               if(len(T)!=lii):         
                  idd=(int(solution[pos+y:pos+z],2)) % (C(len(T),lii)-1)
               else:
                 idd=0   
               
             print('telse=          '+str(len(T)))
             print('liielse=        '+str(lii))
             print('comb(t,lii)else='+str(C(len(T),lii)-1))
             print('iddd  else =         '+str(idd))

             c.append([ac,vMin,idd])
           #q=int(solution[pos+x:pos+y],2) 
             
           Li=c[i][1]
           P=Li
           #print('P='+str(P))
            
           
           pos+=z
           
           activeU=c[i][0]
           id=c[i][2]
             
           print('T= avant la suppression '+str(len(T)))
           print('id='+str(id))
           print('Li='+str(Li))
           if(i==0):# si i==0 on enleve le premier active user de T 
             u=c[0][0]
             
             for v in range(len(T)-1):
               if(u==T[v]):
                  
                  del T[v]
             r=getmyset(Li,len(T),id)

           else:  
             r=getmyset(Li,len(T),id)
           r=list(r)
           l=[]
           
           for n in r:
               
                 l.append(T[n])
               
           #on ajoute les voisin du active user obtenu grace au treillis [[1,[2,3,5,8,9],2,[7,6,4]......]  
           e.append([c[i][0],l]) 
             
           #on supprime les voisins de T
           for n in sorted(r,reverse=True):
               
               del T[n] 

             #print(T)                
        
           print('T= apres la suppression '+str(len(T)))
        
          
           vMax=int(len(T)-((K-(i+2))*(vMin))) 

           print('vMax='+str(vMax)  )
             
          

            

             
         else: break   
  #calcul de la similarité  
  j=sim(m,e,similarity_metric='overlap')
  print('e='+str(e))
  #calcul de la disimilarité    
  g=disim(m,e,similarity_metric='overlap')
  #print('g='+str(g))
                   
  return j+g

def decodagek(kMax,solution):
  pos=len(f"{kMax:b}")
  s=solution[:pos]
  s=int(s,2)
  s=1+(s%(kMax-1))
  return s  


def predict(userId,itemId): 

  reader=Reader(rating_scale=(1, 5))
# Load the movielens-100k dataset
  data = Dataset.load_from_df(ratings, reader)

# Retrieve the trainset.
  trainset, testset = train_test_split(data, test_size=.25)
# Build an algorithm, and train it.

  sim_options = {'name': 'cosine',
               'user_based': True  # compute  similarities between users
               }
  algo = KNNBasic(sim_options=sim_options)
  algo.fit(trainset)
  #predictions = algo.test(testset)
  pred = algo.predict(userId, itemId)

  return pred.est
def generate_recommendation(user_id, metadata, thresh=4):
    
   
    rec=[]
    titles = list(metadata['movieId'].values)
    random.shuffle(titles)
    for title in titles:
        rating = predict(user_id, title)
        if rating >= thresh:
          rec.append(title)
          if(len(rec)==10):
              return rec
"""ul=[1,2,3,4,5] 
r=[]
for i in ul: 
              
      r.append(generate_recommendation(i,ratings))
print(r)"""
